<a href="https://colab.research.google.com/github/Narennnnn/EmoNet-Deep-Learning-for-Facial-Expression-Recognition/blob/main/FacialExpressionDetectionMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import h5py

In [15]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
ls


Dataset.zip  fer2013/  fer2013.csv


In [17]:
%cd drive/MyDrive/Facial\ Expression\ Dataset/

[Errno 2] No such file or directory: 'drive/MyDrive/Facial Expression Dataset/'
/content/drive/MyDrive/Facial Expression Dataset


In [18]:
!unzip Dataset.zip

Archive:  Dataset.zip
replace fer2013.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [19]:
import pandas as pd

csv_path = '/content/drive/MyDrive/Facial Expression Dataset/fer2013.csv'
data = pd.read_csv(csv_path)

image_paths = data['pixels'].values
labels = data['emotion'].values


In [20]:
# Process the image paths
preprocessed_images = []
for image_data in image_paths:
    image = np.array(image_data.split(' '), dtype=np.uint8)
    image = image.reshape((48, 48))
    preprocessed_images.append(image)


In [21]:
# Convert the preprocessed data to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
labels = np.array(labels)


In [22]:
# Convert the preprocessed data to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
labels = np.array(labels)

In [23]:
# Data augmentation using ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [24]:
# Generate augmented images and labels
augmented_images = []
augmented_labels = []
for image, label in zip(preprocessed_images, labels):
    image = np.expand_dims(image, axis=-1)  # Add an additional dimension for the channel
    image = np.expand_dims(image, axis=0)  # Add an additional dimension for the batch
    aug_iter = datagen.flow(image, save_to_dir=None, shuffle=False)
    for aug_image in aug_iter:
        augmented_images.append(aug_image[0])
        augmented_labels.append(label)
        break

augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

In [25]:
# Train-validation-test split
X_train, X_test, y_train, y_test = train_test_split(augmented_images, augmented_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [26]:
# Build the deep learning model
num_classes = len(np.unique(labels))
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [27]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
718/718 [==============================] - 279s 386ms/step - loss: 1.9877 - accuracy: 0.2826 - val_loss: 1.6811 - val_accuracy: 0.3197
Epoch 2/10
718/718 [==============================] - 270s 377ms/step - loss: 1.6081 - accuracy: 0.3697 - val_loss: 1.5889 - val_accuracy: 0.3826
Epoch 3/10
718/718 [==============================] - 271s 378ms/step - loss: 1.5122 - accuracy: 0.4143 - val_loss: 1.5371 - val_accuracy: 0.3925
Epoch 4/10
718/718 [==============================] - 271s 377ms/step - loss: 1.4282 - accuracy: 0.4495 - val_loss: 1.4690 - val_accuracy: 0.4296
Epoch 5/10
718/718 [==============================] - 270s 376ms/step - loss: 1.3514 - accuracy: 0.4793 - val_loss: 1.4357 - val_accuracy: 0.4429
Epoch 6/10
718/718 [==============================] - 268s 373ms/step - loss: 1.2655 - accuracy: 0.5204 - val_loss: 1.4915 - val_accuracy: 0.4472
Epoch 7/10
718/718 [==============================] - 261s 363ms/step - loss: 1.1580 - accuracy: 0.5605 - val_loss: 1.5322 -

In [33]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

225/225 [==============================] - 21s 94ms/step - loss: 1.9998 - accuracy: 0.4283
Test Loss: 1.9998326301574707
Test Accuracy: 0.4282529950141907


In [34]:
# Generate predictions
y_pred = np.argmax(model.predict(X_test), axis=-1)

225/225 [==============================] - 19s 86ms/step


In [35]:
# Generate classification report
target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Angry       0.32      0.39      0.35       985
     Disgust       0.13      0.05      0.07       102
        Fear       0.26      0.21      0.23      1043
       Happy       0.61      0.67      0.64      1765
         Sad       0.34      0.23      0.28      1210
    Surprise       0.54      0.54      0.54       795
     Neutral       0.38      0.45      0.41      1278

    accuracy                           0.43      7178
   macro avg       0.37      0.36      0.36      7178
weighted avg       0.42      0.43      0.42      7178



In [36]:
# Save the model in HDF5 format
model.save('/content/drive/MyDrive/model.h5')